# Query Data Lake - Redshift Spectrum

In this lab, we show you how to query data in your Amazon S3 data lake with Amazon Redshift without loading or moving data. We will also demonstrate how you can leverage views which union data in Redshift Managed storage with data in S3. You can query structured and semi-structured data from files in Amazon S3 without having to copy or move data into Amazon Redshift tables. For latest guide on the file types that can be queried with Redshift Spectrum, please refer to supported data formats.

Objective: Derive data insights to showcase the effect of blizzard on number of taxi rides in January 2016.

Data Set Description: NY city taxi trip data including number of taxi rides by year and month for 3 different taxi companies - fhv, green, and yellow.

Data Set S3 Location:

us-east-1 region - https://s3.console.aws.amazon.com/s3/buckets/redshift-demos?region=us-east-1&prefix=data/NY-Pub/ 

us-west-2 region - https://s3.console.aws.amazon.com/s3/buckets/us-west-2.serverless-analytics?prefix=canonical/NY-Pub/ 

Below is an overview of the use case steps involved in this lab. 

![](https://static.us-east-1.prod.workshops.aws/public/731bd8a6-b5d4-4391-9b55-7cfb7a6d8abc/static/images/lab4/usecase.png)

## Create and run Glue crawler to populate Glue data catalog

In this part of the lab, we will perform following activities:

- Query historical data residing on S3 by creating an external DB for Redshift Spectrum.
- Introspect the historical data, perhaps rolling-up the data in novel ways to see trends over time, or other dimensions.

Note the partitioning scheme is Year, Month, Type (where Type is a taxi company). Here's a Screenshot:

![](https://static.us-east-1.prod.workshops.aws/public/731bd8a6-b5d4-4391-9b55-7cfb7a6d8abc/static/images/lab4/partitions.png)

### Create external schema (and DB) for Redshift Spectrum

You can create an external table in Amazon Redshift, AWS Glue, Amazon Athena, or an Apache Hive metastore. If your external table is defined in AWS Glue, Athena, or a Hive metastore, you first create an external schema that references the external database. Then, you can reference the external table in your SELECT statement by prefixing the table name with the schema name, without needing to create the table in Amazon Redshift.

In this lab, you will use AWS Glue Crawler to create external table adb305.ny_pub stored in parquet format under location s3://us-west-2.serverless-analytics/canonical/NY-Pub/.

1. Navigate to the Glue Crawler Page.
1. Click on Create Crawler, and enter the crawler name NYTaxiCrawler and click Next.
1. Click on Add a data source.
1. Choose S3 as the data store, select In a different account and enter S3 file path `s3://redshift-demos/data/NY-Pub for us-east-1` and `s3://us-west-2.serverless-analytics/canonical/NY-Pub` for us-west-2. Click on Add an S3 data source.
1. Select the S3 as data source for crawler and click Next
1. Click Create new IAM role and click Next
1. Enter AWSGlueServiceRole-RedshiftImmersion and click Create
1. Click on Add database and enter Name spectrumdb
1. Go back to Glue Console, refresh the target database and select spectrumdb
1. Select all remaining defaults and click Create crawler. Select the crawler - NYTaxiCrawler and click Run.
1. After Crawler run completes, you can see a new table ny_pub in Glue Catalog

### Create external schema adb305 in Redshift and select from Glue catalog table - ny_pub

1. Go to Redshift console.
1. Click on Serverless dashboard menu item to the left side of the console. Click on the name space provisioned earlier. Click Query data.
1. Create an external schema adb305 pointing to your Glue Catalog Database spectrumdb.

```sql
CREATE external SCHEMA adb305
FROM data catalog DATABASE 'spectrumdb'
IAM_ROLE default
CREATE external DATABASE if not exists;
```

**Pin-point the Blizzard**

You can query the table ny_pub, defined in Glue Catalog from Redshift external schema. In January 2016, there is a date which had the lowest number of taxi rides due to a blizzard. Can you find that date?

```sql
SELECT TO_CHAR(pickup_datetime, 'YYYY-MM-DD'),COUNT(*)
FROM adb305.ny_pub
WHERE YEAR = 2016 and Month = 01
GROUP BY 1
ORDER BY 2;
```

### Create internal schema workshop_das

Create a schema workshop_das for tables that will reside on the Redshift Managed Storage.

```sql
CREATE SCHEMA workshop_das;
```

### Run CTAS to create and load Redshift table workshop_das.taxi_201601 by selecting from external table

Create table workshop_das.taxi_201601 to load data for green taxi company for January 2016

```sql
CREATE TABLE workshop_das.taxi_201601 AS
SELECT *
FROM adb305.ny_pub
WHERE year = 2016 AND month = 1 AND type = 'green';
```

Note: What about column compression/encoding? Remember that on a CTAS, Amazon Redshift automatically assigns compression encoding as follows:

- Columns that are defined as sort keys are assigned RAW compression.
- Columns that are defined as BOOLEAN, REAL, or DOUBLE PRECISION, or GEOMETRY data types are assigned RAW compression.
- Columns that are defined as SMALLINT, INTEGER, BIGINT, DECIMAL, DATE, TIMESTAMP, or TIMESTAMPTZ are assigned AZ64 compression.
- Columns that are defined as CHAR or VARCHAR are assigned LZO compression.

https://docs.aws.amazon.com/redshift/latest/dg/r_CTAS_usage_notes.html 

```sql
ANALYZE COMPRESSION workshop_das.taxi_201601;
```

- Add to the taxi_201601 table with an INSERT/SELECT statement for other taxi companies.

```sql
INSERT INTO workshop_das.taxi_201601 (
SELECT *
FROM adb305.ny_pub
WHERE year = 2016 AND month = 1 AND type != 'green');
```

### Drop 201601 partitions from external table

Now that we've loaded all January, 2016 data, we can remove the partitions from the Spectrum table so there is no overlap between the Redshift Managed Storage (RMS) table and the Spectrum table.

```sql
ALTER TABLE adb305.ny_pub DROP PARTITION(year=2016, month=1, type='fhv');
ALTER TABLE adb305.ny_pub DROP PARTITION(year=2016, month=1, type='green');
ALTER TABLE adb305.ny_pub DROP PARTITION(year=2016, month=1, type='yellow');
```

### Create combined view public.adb305_view_NY_TaxiRides

```sql
CREATE VIEW adb305_view_NYTaxiRides AS
  SELECT * FROM workshop_das.taxi_201601
  UNION ALL
  SELECT * FROM adb305.ny_pub
WITH NO SCHEMA BINDING;
```

**Explain displays the execution plan for a query statement without running the query**
- Note the use of the partition columns in the SELECT and WHERE clauses. Where were those columns in your Spectrum table definition?
- Note the filters being applied either at the partition or file levels in the Spectrum dataset of the query (versus the Redshift Managed Storage dataset).
- If you actually run the query (and not just generate the explain plan), does the runtime surprise you? Why or why not?

```sql
EXPLAIN
SELECT year, month, type, COUNT(*)
FROM adb305_view_NYTaxiRides
WHERE year = 2016 AND month IN (1,2) AND passenger_count = 4
GROUP BY 1,2,3 ORDER BY 1,2,3;
```

![](https://static.us-east-1.prod.workshops.aws/public/731bd8a6-b5d4-4391-9b55-7cfb7a6d8abc/static/images/lab4/result3.png)

Note the S3 Seq Scan was run against the data on Amazon S3. The S3 Seq Scan node shows the Filter: (passenger_count = 4) was processed in the Redshift Spectrum layer.

For ways to improve Redshift Spectrum performance, please refer to https://docs.aws.amazon.com/redshift/latest/dg/c-spectrum-external-performance.html 